In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

from sklearn.metrics.pairwise import cosine_similarity

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stadia-steam-recommendataion/steamspy_tag_data.csv
/kaggle/input/stadia-steam-recommendataion/steam-200k.csv
/kaggle/input/stadia-steam-recommendataion/steam_description_data.csv
/kaggle/input/stadia-steam-recommendataion/steam.csv


In [36]:
df = pd.read_csv('/kaggle/input/stadia-steam-recommendataion/steam.csv')
df.head(10)

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
5,60,Ricochet,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Valve Anti-Ch...,Action,Action;FPS;Multiplayer,0,2758,684,175,10,5000000-10000000,3.99
6,70,Half-Life,1998-11-08,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Online Multi-Player...,Action,FPS;Classic;Action,0,27755,1100,1300,83,5000000-10000000,7.19
7,80,Counter-Strike: Condition Zero,2004-03-01,1,Valve,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,Action;FPS;Multiplayer,0,12120,1439,427,43,10000000-20000000,7.19
8,130,Half-Life: Blue Shift,2001-06-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player,Action,FPS;Action;Sci-fi,0,3822,420,361,205,5000000-10000000,3.99
9,220,Half-Life 2,2004-11-16,1,Valve,Valve,windows;mac;linux,0,Single-player;Steam Achievements;Steam Trading...,Action,FPS;Action;Sci-fi,33,67902,2419,691,402,10000000-20000000,7.19


In [37]:
df.shape

(27075, 18)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

In [39]:
df.duplicated().sum()

0

In [40]:
df.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [41]:
df.drop(['release_date', 'english', 'developer', 'publisher', 'required_age', 'categories', 'achievements', 'average_playtime',
        'negative_ratings', 'median_playtime', 'owners', 'price'], axis=1, inplace=True)

In [42]:
df.head()

,appid,name,platforms,genres,steamspy_tags,positive_ratings
0,10,Counter-Strike,windows;mac;linux,Action,Action;FPS;Multiplayer,124534
1,20,Team Fortress Classic,windows;mac;linux,Action,Action;FPS;Multiplayer,3318
2,30,Day of Defeat,windows;mac;linux,Action,FPS;World War II;Multiplayer,3416
3,40,Deathmatch Classic,windows;mac;linux,Action,Action;FPS;Multiplayer,1273
4,50,Half-Life: Opposing Force,windows;mac;linux,Action,FPS;Action;Sci-fi,5250


In [43]:
df['positive_ratings'] = df.positive_ratings / max(df.positive_ratings)

In [46]:
desc_df = pd.read_csv('/kaggle/input/stadia-steam-recommendataion/steam_description_data.csv')
desc_df.head()

,steam_appid,detailed_description,about_the_game,short_description
0,10,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [50]:
df_final = pd.merge(df, desc_df, left_on='appid', right_on='steam_appid', how='inner')
df_final.drop(['steam_appid', 'about_the_game', 'short_description', 'positive_ratings', 'platforms'], axis=1, inplace=True)
df_final.head(10)

,appid,name,genres,steamspy_tags,detailed_description
0,10,Counter-Strike,Action,Action;FPS;Multiplayer,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,Action,Action;FPS;Multiplayer,One of the most popular online action games of...
2,30,Day of Defeat,Action,FPS;World War II;Multiplayer,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,Action,Action;FPS;Multiplayer,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,Action,FPS;Action;Sci-fi,Return to the Black Mesa Research Facility as ...
5,60,Ricochet,Action,Action;FPS;Multiplayer,A futuristic action game that challenges your ...
6,70,Half-Life,Action,FPS;Classic;Action,Named Game of the Year by over 50 publications...
7,80,Counter-Strike: Condition Zero,Action,Action;FPS;Multiplayer,"With its extensive Tour of Duty campaign, a ne..."
8,130,Half-Life: Blue Shift,Action,FPS;Action;Sci-fi,Made by Gearbox Software and originally releas...
9,220,Half-Life 2,Action,FPS;Action;Sci-fi,1998. HALF-LIFE sends a shock through the game...


In [52]:
df_final.steamspy_tags = df_final.steamspy_tags.apply(lambda x: " ".join(x.split(";")))
df_final.head(10)

,appid,name,genres,steamspy_tags,detailed_description
0,10,Counter-Strike,Action,Action FPS Multiplayer,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,Action,Action FPS Multiplayer,One of the most popular online action games of...
2,30,Day of Defeat,Action,FPS World War II Multiplayer,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,Action,Action FPS Multiplayer,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,Action,FPS Action Sci-fi,Return to the Black Mesa Research Facility as ...
5,60,Ricochet,Action,Action FPS Multiplayer,A futuristic action game that challenges your ...
6,70,Half-Life,Action,FPS Classic Action,Named Game of the Year by over 50 publications...
7,80,Counter-Strike: Condition Zero,Action,Action FPS Multiplayer,"With its extensive Tour of Duty campaign, a ne..."
8,130,Half-Life: Blue Shift,Action,FPS Action Sci-fi,Made by Gearbox Software and originally releas...
9,220,Half-Life 2,Action,FPS Action Sci-fi,1998. HALF-LIFE sends a shock through the game...


In [54]:
df_final['description'] = df_final.steamspy_tags +" "+ df_final.detailed_description
df_final.name = df_final.name.apply(lambda x: x.lower())
df_final.description = df_final.description.apply(lambda x: x.lower())
df_final.head(10)

,appid,name,genres,steamspy_tags,detailed_description,description
0,10,counter-strike,Action,Action FPS Multiplayer,Play the world's number 1 online action game. ...,action fps multiplayer play the world's number...
1,20,team fortress classic,Action,Action FPS Multiplayer,One of the most popular online action games of...,action fps multiplayer one of the most popular...
2,30,day of defeat,Action,FPS World War II Multiplayer,Enlist in an intense brand of Axis vs. Allied ...,fps world war ii multiplayer enlist in an inte...
3,40,deathmatch classic,Action,Action FPS Multiplayer,Enjoy fast-paced multiplayer gaming with Death...,action fps multiplayer enjoy fast-paced multip...
4,50,half-life: opposing force,Action,FPS Action Sci-fi,Return to the Black Mesa Research Facility as ...,fps action sci-fi return to the black mesa res...
5,60,ricochet,Action,Action FPS Multiplayer,A futuristic action game that challenges your ...,action fps multiplayer a futuristic action gam...
6,70,half-life,Action,FPS Classic Action,Named Game of the Year by over 50 publications...,fps classic action named game of the year by o...
7,80,counter-strike: condition zero,Action,Action FPS Multiplayer,"With its extensive Tour of Duty campaign, a ne...",action fps multiplayer with its extensive tour...
8,130,half-life: blue shift,Action,FPS Action Sci-fi,Made by Gearbox Software and originally releas...,fps action sci-fi made by gearbox software and...
9,220,half-life 2,Action,FPS Action Sci-fi,1998. HALF-LIFE sends a shock through the game...,fps action sci-fi 1998. half-life sends a shoc...


In [55]:
df_final.drop(['genres', 'steamspy_tags', 'detailed_description'], axis=1, inplace=True)
df_final.head(10)

,appid,name,description
0,10,counter-strike,action fps multiplayer play the world's number...
1,20,team fortress classic,action fps multiplayer one of the most popular...
2,30,day of defeat,fps world war ii multiplayer enlist in an inte...
3,40,deathmatch classic,action fps multiplayer enjoy fast-paced multip...
4,50,half-life: opposing force,fps action sci-fi return to the black mesa res...
5,60,ricochet,action fps multiplayer a futuristic action gam...
6,70,half-life,fps classic action named game of the year by o...
7,80,counter-strike: condition zero,action fps multiplayer with its extensive tour...
8,130,half-life: blue shift,fps action sci-fi made by gearbox software and...
9,220,half-life 2,fps action sci-fi 1998. half-life sends a shoc...


In [62]:
stop_words = stopwords.words('english')
stem = PorterStemmer()
df_final.description = df_final.description.astype('str')
df_final.description = df_final.description.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x))
df_final.description = df_final.description.apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
df_final.description = df_final.description.apply(lambda x: stem.stem(x))

In [66]:
cv = CountVectorizer(tokenizer=nltk.word_tokenize, ngram_range=(1,4), max_features = 2048)
vectors = cv.fit_transform(df_final.description).toarray()
similarity = cosine_similarity(vectors)

In [113]:
def recommend(game_name):
    game_index = df_final[df_final.name == game_name].index[0]
    distances = similarity[game_index]
    games_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x: x[1])[1:10]
    print('Here are games you may like!')
    print('\n')
    for i in games_list:
        
        print(df_final.iloc[i[0]]['name'])
    

In [117]:
recommend('call of duty® 4: modern warfare®')

Here are games you may like!


call of duty®: modern warfare® 2
tom clancy's ghost recon®
call of duty®: black ops
crysis
call of duty: world at war
delta force 2
assassin’s creed® brotherhood
homefront
silent hunter®: wolves of the pacific
